# TMDB

In [1]:
import json
with open(r"C:\Users\nbeac\.secret\tmdb_api.json", "r") as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [2]:
import tmdbsimple as tmdb
tmdb.API_KEY = login['api-key']

In [4]:
import os, time
import pandas as pd

In [5]:
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title_akas_cleaned.csv.gz',
 'title_basics_cleaned.csv.gz',
 'title_ratings_cleaned.csv.gz']

In [6]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv("Data/title_basics_cleaned.csv.gz")
basics.info()
basics.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935 entries, 0 to 2934
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          2935 non-null   object 
 1   titleType       2935 non-null   object 
 2   primaryTitle    2935 non-null   object 
 3   originalTitle   2935 non-null   object 
 4   isAdult         2935 non-null   int64  
 5   startYear       2935 non-null   float64
 6   runtimeMinutes  2935 non-null   int64  
 7   genres          2935 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 183.6+ KB


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,118,"Comedy,Fantasy,Romance"
1,tt0113026,movie,The Fantasticks,The Fantasticks,0,2000.0,86,"Musical,Romance"
2,tt0113092,movie,For the Cause,For the Cause,0,2000.0,100,"Action,Adventure,Drama"
3,tt0114447,movie,The Silent Force,The Silent Force,0,2001.0,90,Action
4,tt0115937,movie,Consequence,Consequence,0,2000.0,91,Drama


In [7]:
def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

# API Extraction Function

In [8]:
def get_movie_with_rating(movie_id):
    #Get the movie object for the current id
    movie = imdb.Movies(movie_id)
    # save the .info .releases dictionaries
    movie_info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        if c['iso_3166_1'] =='US':
            movie_info['certification'] = c['certification']
    return movie_info

# Before the Loops

In [15]:
YEARS_TO_GET = list(range(2000,2002))
YEARS_TO_GET

[2000, 2001, 2002]

# OUTER Loop

## Setting up a Progress Bar

Select a JSON_FILE filename to save the results in progress.
* Check if the file exists.
    * if no:
        * Create the empy JSON_FILE with "with open" that just contains the key "imdb_id"
            
    * if yes:
        * Do nothing!

The code below will identify the folder in the FOLDER we just define above and will name the file based on the current year

In [12]:
# OUTER LOOP
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):

    
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    basics = pd.read_csv("Data/title_basics_cleaned.csv.gz")
    df = basics.loc[basics['startYear']==YEAR].copy()
    movie_ids = df['tconst'].copy()
    
    previous_df = pd.read_json(JSON_FILE)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

# INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)

        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            continue


    
    final_year_df = pd.read_json(JSON_FILE)
    
    csv_fname = f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz"
    final_year_df.to_csv(csv_fname, compression="gzip", index=False)
    

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1409 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1526 [00:00<?, ?it/s]

In [13]:
tmdb_results_2000 = pd.read_csv(r"C:\Users\nbeac\OneDrive\Documents\GitHub\Dojo-Project-03\Data\final_tmdb_data_2000.csv.gz")
tmdb_results_2000.head()

,imdb_id
0,0


In [14]:
tmdb_results_2001 = pd.read_csv(r"C:\Users\nbeac\OneDrive\Documents\GitHub\Dojo-Project-03\Data\final_tmdb_data_2001.csv.gz")
tmdb_results_2001.head()

,imdb_id
0,0
